# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import os

import azureml.core
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core import Environment
from azureml.core import ScriptRunConfig

from azureml.core import Dataset

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:

ws = Workspace.from_config()
ws.write_config(".azureml")

experiment_name = 'udacity-capstone-project'
project_folder = "hyperdrive-heartfailure"

experiments = Experiment.list(ws, experiment_name=experiment_name)
if not experiments:
      experiment = Experiment(workspace=ws, name=experiment_name)
else:
      experiment = experiments[0]


print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cpu_cluster_name = "cpu-cluster"
vm_size = "Standard_D3_V2"
max_nodes = 6

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cluster. use it")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                           max_nodes=max_nodes)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# a detailed status for the current cluster.
print(compute_target.get_status().serialize())

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
import os
import shutil

from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.parameter_expressions import uniform, choice

from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.runconfig import HyperDriveConfig

In [ ]:
# Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.2, evaluation_interval=50, delay_evaluation=5) #<your policy here>

#TODO: Create the different params that you will be using during training
parameter_space = { "--C": uniform(0.1, 1.0), 
                    "--max_iter": choice([50, 100, 500, 1000]) }
param_sampling = RandomParameterSampling(parameter_space)



In [ ]:
script_folder = "training"
os.makedirs(os.path.join(".", script_folder), exist_ok=True)

# Create a SKLearn estimator for use with train.py
shutil.copy("./train.py", os.path.join(".", script_folder))
source_directory=os.path.join(".", script_folder)

In [ ]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- numpy
- pandas
- scikit-learn
- pip:
  - azureml-defaults

In [ ]:
from azureml.core import Environment

sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

In [ ]:
# Use ScrptRunConfig
src = ScriptRunConfig(source_directory=source_directory,
                      script='train.py',
                      arguments=[],
                      compute_target=compute_target,
                      environment=sklearn_env)

#Create your estimator and hyperdrive config
# estimator = <your estimator here>

# hyperdrive_run_config = <your config here?

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_run_config = HyperDriveConfig(
                        run_config=src,
                        hyperparameter_sampling=param_sampling,
                        policy=early_termination_policy,
                        primary_metric_name="accuracy",
                        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                        max_total_runs=25,
                        max_concurrent_runs=max_nodes-1
                        ) 

In [ ]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service